In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean
import itertools

sizes = [10]
instances = ["5", "6b", "7"]
sets = ['IN']
solvers = ['bb']

In [14]:
for (size, selected_set, instance) in itertools.product(sizes, sets, instances):
    !echo "./{selected_set}/graf_{size}_{instance}.txt"
    _return_val = !cmake-build-debug/pdp --source "./{selected_set}/graf_{size}_{instance}.txt"

./IN/graf_10_5.txt
./IN/graf_10_6b.txt
./IN/graf_10_7.txt


In [10]:
class Solutions:

    def __init__(self,solver, selected_set, size, times, counts, errors):
        self.solver = solver
        self.set=selected_set
        self.size = size
        self.times = times
        self.counts = counts
        self.errors = errors

    def get_time_max(self):
        return max(self.times)

    def get_count_max(self):
        return max(self.counts)

    def get_error_max(self):
        return max(self.errors)

    def get_time_mean(self):
        return mean(self.times)

    def get_count_mean(self):
        return mean(self.counts)

    def get_error_mean(self):
        return mean(self.errors)

def getData(solver_names, sets, nums):
    solutions = []

    for (solver_name, set, num) in itertools.product(solver_names, sets, nums):

        filename = 'OUT/'+solver_name+'/'+set+'/'+set+str(num)+'_inst.dat'
        filename_ref = set+'/'+set+''+str(num)+'_sol.dat'

        data = pd.read_csv(filename, sep=' ', header=None).values
        data_ref = pd.read_csv(filename_ref, sep=' ', header=None).values
        
        ids = data[:,0]
        ids_ref = data_ref[:,0]
        times = data[:,-1]
        counts = data[:,-2]
        values = data[:,3]
        values_ref = data_ref[:,2]

        errors = []
        current_id = 1
        for i in range(0,len(ids_ref)):
            if current_id == ids_ref[i]:
                if(values_ref[i] == 0):
                    errors.append(0)
                else:
                    errors.append( (values_ref[i] - values[current_id-1])/values_ref[i] )
                
                current_id+=1

                if current_id > len(ids):
                    break

        solutions.append(Solutions(solver_name, set, num,times,counts,errors))

    return solutions

def listPlot(ax, data, solvers, set, sizes, colors, param):
    filtered_data = tuple(filter(lambda x: x.set == set, data))
    
    for (solver,color) in zip(solvers,colors):
        solver_data = tuple(filter(lambda x: x.solver == solver, filtered_data))

        if param == 'time':
            ax.plot( sizes, tuple(map(lambda x: x.get_time_max(),solver_data)),linestyle=':', label="max "+solver, c=color)
            ax.plot( sizes, tuple(map(lambda x: x.get_time_mean(),solver_data)), label="mean "+solver,c=color)

        if param == 'count':
            ax.plot( sizes, tuple(map(lambda x: x.get_count_max(),solver_data)),linestyle=':', label="max "+solver, c=color)
            ax.plot( sizes, tuple(map(lambda x: x.get_count_mean(),solver_data)), label="mean "+solver,c=color) 

        if param == 'error':
            ax.plot( sizes, tuple(map(lambda x: x.get_error_max(),solver_data)),linestyle=':', label="max "+solver, c=color)
            ax.plot( sizes, tuple(map(lambda x: x.get_error_mean(),solver_data)), label="mean "+solver,c=color)

    if param == 'time':
        ax.set(ylabel='time (s)',title='mean, max times ('+set+')')
    if param == 'count':
        ax.set(ylabel='count',title='mean, max counts ('+set+')')
    if param == 'error':
        ax.set(ylabel='relative error',title='mean, max errors ('+set+')')

    ax.legend()

def histogram(ax, data, solvers, set, size, colors, param, bins):
    filtered_data = tuple(filter(lambda x: x.set == set and x.size == size, data))

    to_plot = []
    labels =[]
    for solver in solvers:
        solver_data = tuple(filter(lambda x: x.solver == solver, filtered_data))

        if param == 'time':
            to_plot.append(tuple(map(lambda x: x.times,solver_data))[0])
            labels.append(solver)

        if param == 'count':
            to_plot.append(tuple(map(lambda x: x.counts,solver_data))[0])
            labels.append(solver)

        if param == 'error':
            to_plot.append(tuple(map(lambda x: x.errors,solver_data))[0])
            labels.append(solver)

    ax.hist(to_plot, bins, color=colors, alpha=0.5, label=labels)

    if param == 'time':
        ax.set(xlabel='time (s)',title='time ('+set+str(size)+')')

    if param == 'count':
        ax.set(xlabel='count',title='counts ('+set+str(size)+')')

    if param == 'error':
        ax.set(xlabel='relative error',title='errors ('+set+str(size)+')')

    ax.legend()

data = getData( solvers, sets, sizes )

FileNotFoundError: [Errno 2] No such file or directory: 'OUT/bb/IN/IN10_inst.dat'

In [ ]:
"""fig, ax = plt.subplots(1,2)
fig.set_figwidth(14)
listPlot(ax[0],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'NK', sizes, ['red','blue', 'green'], 'time')
listPlot(ax[1],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'ZKC', sizes, ['red','blue', 'green'], 'time')

plt.savefig('OUT/fptasTimeList.pdf', bbox_inches='tight')

fig, ax = plt.subplots(1,2)
fig.set_figwidth(14)
listPlot(ax[0],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'NK', sizes, ['red','blue', 'green'], 'error')
listPlot(ax[1],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'ZKC', sizes, ['red','blue', 'green'], 'error')

plt.savefig('OUT/fptasErrorList.pdf', bbox_inches='tight')

fig, ax = plt.subplots(1,2)
fig.set_figwidth(14)
histogram(ax[0],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'NK', 30, ['red','blue', 'green'], 'time', 20)
histogram(ax[1],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'ZKC', 30, ['red','blue', 'green'], 'time', 20)

plt.savefig('OUT/fptasTimeHist.pdf', bbox_inches='tight')

fig, ax = plt.subplots(1,2)
fig.set_figwidth(14)
histogram(ax[0],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'NK', 30, ['red','blue', 'green'], 'error', 20)
histogram(ax[1],data,['FPTAS1', 'FPTAS2', 'FPTAS3'],'ZKC', 30, ['red','blue', 'green'], 'error', 20)

plt.savefig('OUT/fptasErrorHist.pdf', bbox_inches='tight')

plt.show()"""